In this notebook I run statistical learning models on residential home sales in the Bell school area of Chicago, a highly gentrifying neighborhood with a large number of flipping and teardowns, to demonstrate that models with no inherent bias in their method lead to biased results, overvaluing low priced homes and undervaluing high priced homes. This is not surprising since it is a basic property of models based on averaging. The problem, regression towards the mean, is so intrinsic to these methods that the basic method bares its name. This property exasperates inequality as it leads to lesser taxation of wealthy people and higher taxation on poor people. The results could be used to advocate for a transition from property tax to income tax as was recently attempted in Illinois.  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.decomposition import PCA
#from library.sb_utils import save_file
import statsmodels.api as sm 
from statsmodels.graphics.api import abline_plot 
import warnings 
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
import pickle
from sklearn import __version__ as sklearn_version
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
bell_sales = pd.read_csv('../data/bell_sales_cleaned_mhomes.csv')

In [3]:
bell_sales.head()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
0,0,1011000050000,2014.0,205,205000.0,6534.0,112.0,1.0,1.0,6.0,...,2.0,1.0,2.0,1.0,0.0,3.0,1941.0,142.0,-88.139379,42.153953
1,1,1011000180000,2020.0,205,307500.0,12012.0,127.0,1.0,1.0,7.0,...,1.0,1.0,3.0,2.0,0.0,3.0,2000.0,290.0,-88.140524,42.153505
2,2,1011000190000,2020.0,202,345000.0,6699.0,80.0,1.0,1.0,4.0,...,1.0,1.0,3.0,1.0,0.0,4.0,807.0,171.0,-88.140083,42.153577
3,3,1011000420000,2020.0,205,375000.0,9036.0,109.0,1.0,1.0,7.0,...,1.0,1.0,3.0,2.0,1.0,5.0,1408.0,351.0,-88.138970,42.153064
4,4,1011000430000,2017.0,203,370000.0,8712.0,82.0,1.0,1.0,6.0,...,2.0,1.0,2.0,1.0,0.0,1.0,1366.0,178.0,-88.138708,42.153038


In [4]:
bell_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296509 entries, 0 to 296508
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             296509 non-null  int64  
 1   meta_pin               296509 non-null  int64  
 2   meta_year              296509 non-null  float64
 3   meta_class             296509 non-null  int64  
 4   meta_sale_price        296509 non-null  float64
 5   char_hd_sf             296509 non-null  float64
 6   char_age               296509 non-null  float64
 7   char_ext_wall          296405 non-null  float64
 8   char_roof_cnst         296404 non-null  float64
 9   char_rooms             296509 non-null  float64
 10  char_beds              296509 non-null  float64
 11  char_bsmt              296417 non-null  float64
 12  char_bsmt_fin          296416 non-null  float64
 13  char_heat              296403 non-null  float64
 14  char_air               296420 non-nu

In [5]:
bell_sales = bell_sales.dropna()

In [6]:
bell_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295916 entries, 0 to 296508
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             295916 non-null  int64  
 1   meta_pin               295916 non-null  int64  
 2   meta_year              295916 non-null  float64
 3   meta_class             295916 non-null  int64  
 4   meta_sale_price        295916 non-null  float64
 5   char_hd_sf             295916 non-null  float64
 6   char_age               295916 non-null  float64
 7   char_ext_wall          295916 non-null  float64
 8   char_roof_cnst         295916 non-null  float64
 9   char_rooms             295916 non-null  float64
 10  char_beds              295916 non-null  float64
 11  char_bsmt              295916 non-null  float64
 12  char_bsmt_fin          295916 non-null  float64
 13  char_heat              295916 non-null  float64
 14  char_air               295916 non-nu

In [7]:
bell_sales.describe()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
count,295916.000000,2.959160e+05,295916.000000,295916.000000,2.959160e+05,2.959160e+05,295916.000000,295916.000000,295916.000000,295916.000000,...,295916.000000,295916.000000,295916.000000,295916.000000,295916.000000,295916.000000,295916.000000,295916.000000,295916.000000,295916.000000
mean,171491.373393,1.636984e+13,2016.517620,221.986432,3.035447e+05,7.597800e+03,61.017870,1.983242,1.118250,6.277484,...,1.448783,0.378229,2.479058,1.521131,0.512733,3.200236,1628.484888,193.884741,-87.796320,41.855196
std,105260.434899,8.629276e+12,2.254982,31.929445,3.125274e+05,1.256604e+04,28.996434,0.816677,0.579243,1.599699,...,0.497371,0.605527,0.787632,0.723228,0.543848,1.643556,805.426046,99.937828,0.150952,0.181549
min,0.000000,1.011000e+12,2013.000000,202.000000,1.000000e+02,2.450000e+02,1.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,400.000000,2.000000,-88.263506,41.469868
25%,74926.750000,9.281240e+12,2015.000000,203.000000,1.400000e+05,3.780000e+03,45.000000,1.000000,1.000000,5.000000,...,1.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1101.000000,115.000000,-87.867046,41.712811
50%,167195.500000,1.534209e+13,2017.000000,204.000000,2.295000e+05,6.089000e+03,60.000000,2.000000,1.000000,6.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,3.000000,1369.000000,201.000000,-87.774068,41.875178
75%,268704.250000,2.412422e+13,2018.000000,234.000000,3.570000e+05,8.616000e+03,76.000000,3.000000,1.000000,7.000000,...,2.000000,1.000000,3.000000,2.000000,1.000000,3.000000,1910.000000,278.000000,-87.693359,42.016015
max,346292.000000,3.332302e+13,2020.000000,295.000000,1.270000e+07,2.980767e+06,184.000000,4.000000,6.000000,64.000000,...,2.000000,9.000000,3.000000,42.000000,7.000000,8.000000,17357.000000,366.000000,-87.524890,42.153979


In [8]:
bell_sales = bell_sales[(bell_sales.meta_sale_price>bell_sales.meta_sale_price.quantile(0.005))&(bell_sales.meta_sale_price<bell_sales.meta_sale_price.quantile(0.995))]

In [9]:
bell_sales.describe()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
count,292949.000000,2.929490e+05,292949.000000,292949.000000,2.929490e+05,2.929490e+05,292949.000000,292949.000000,292949.000000,292949.000000,...,292949.000000,292949.000000,292949.000000,292949.000000,292949.000000,292949.000000,292949.000000,292949.000000,292949.000000,292949.000000
mean,171270.957883,1.635623e+13,2016.523883,222.056061,2.929421e+05,7.558357e+03,60.972869,1.985373,1.112252,6.259567,...,1.448389,0.370689,2.480210,1.510406,0.510369,3.190487,1615.025957,193.960113,-87.797558,41.855323
std,105254.242069,8.629396e+12,2.254396,31.977400,2.538731e+05,1.238805e+04,28.773080,0.816977,0.566317,1.561645,...,0.497330,0.585672,0.787281,0.698005,0.540013,1.635473,765.234436,99.918595,0.150987,0.181534
min,0.000000,1.011000e+12,2013.000000,202.000000,9.501000e+03,2.450000e+02,1.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,400.000000,2.000000,-88.263506,41.469868
25%,74822.000000,9.281160e+12,2015.000000,203.000000,1.400000e+05,3.780000e+03,45.000000,1.000000,1.000000,5.000000,...,1.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1102.000000,115.000000,-87.868762,41.712996
50%,166982.000000,1.534117e+13,2017.000000,204.000000,2.295000e+05,6.100000e+03,60.000000,2.000000,1.000000,6.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,3.000000,1368.000000,201.000000,-87.775392,41.875278
75%,268514.000000,2.412321e+13,2018.000000,234.000000,3.550000e+05,8.612000e+03,76.000000,3.000000,1.000000,7.000000,...,2.000000,1.000000,3.000000,2.000000,1.000000,3.000000,1902.000000,278.000000,-87.695018,42.016105
max,346292.000000,3.332302e+13,2020.000000,295.000000,1.905188e+06,2.980767e+06,184.000000,4.000000,6.000000,64.000000,...,2.000000,9.000000,3.000000,42.000000,7.000000,8.000000,13144.000000,366.000000,-87.524890,42.153979


In [10]:
bell_sales = bell_sales[(bell_sales.char_bldg_sf>bell_sales.char_bldg_sf.quantile(0.005))&(bell_sales.char_bldg_sf<bell_sales.char_bldg_sf.quantile(0.995))]

In [11]:
bell_sales.describe()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
count,289839.000000,2.898390e+05,289839.000000,289839.000000,2.898390e+05,2.898390e+05,289839.000000,289839.000000,289839.000000,289839.000000,...,289839.000000,289839.000000,289839.000000,289839.000000,289839.000000,289839.000000,289839.000000,289839.000000,289839.000000,289839.000000
mean,171464.426354,1.637514e+13,2016.522500,222.225291,2.899623e+05,7.401352e+03,60.895145,1.988104,1.107053,6.248469,...,1.447676,0.364492,2.480860,1.501434,0.509338,3.176708,1599.837282,193.926183,-87.797425,41.854966
std,105244.714550,8.627857e+12,2.254343,32.087098,2.468626e+05,1.165647e+04,28.601977,0.816597,0.553029,1.515682,...,0.497256,0.571720,0.787449,0.677041,0.537245,1.622168,702.885769,99.889917,0.150870,0.181564
min,0.000000,1.011000e+12,2013.000000,202.000000,9.501000e+03,2.450000e+02,1.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,673.000000,2.000000,-88.263506,41.469868
25%,74963.500000,9.283020e+12,2015.000000,203.000000,1.400000e+05,3.780000e+03,45.000000,1.000000,1.000000,5.000000,...,1.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1104.000000,115.000000,-87.868582,41.712735
50%,167043.000000,1.534126e+13,2017.000000,204.000000,2.291000e+05,6.100000e+03,60.000000,2.000000,1.000000,6.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,3.000000,1368.000000,201.000000,-87.775320,41.875108
75%,268748.500000,2.412438e+13,2018.000000,234.000000,3.530000e+05,8.580000e+03,75.000000,3.000000,1.000000,7.000000,...,2.000000,1.000000,3.000000,2.000000,1.000000,3.000000,1895.000000,278.000000,-87.694965,42.015902
max,346292.000000,3.332302e+13,2020.000000,295.000000,1.905000e+06,2.980767e+06,184.000000,4.000000,6.000000,64.000000,...,2.000000,9.000000,3.000000,42.000000,7.000000,8.000000,4823.000000,366.000000,-87.524890,42.153979


In [12]:
bell_sales = bell_sales[(bell_sales.char_hd_sf>bell_sales.char_hd_sf.quantile(0.005))&(bell_sales.char_hd_sf<bell_sales.char_hd_sf.quantile(0.995))]

In [13]:
bell_sales.describe()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
count,286935.000000,2.869350e+05,286935.000000,286935.000000,2.869350e+05,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000,...,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000,286935.000000
mean,171738.977897,1.640128e+13,2016.522509,221.866235,2.884860e+05,6966.502678,61.149114,1.987781,1.102023,6.242595,...,1.450506,0.359834,2.477979,1.496538,0.507247,3.170080,1594.128771,193.933079,-87.796878,41.854492
std,105160.894712,8.616713e+12,2.254252,31.761549,2.463899e+05,4900.266183,28.544146,0.817708,0.544855,1.510581,...,0.497545,0.566890,0.788939,0.674261,0.536805,1.616796,696.327156,99.887625,0.149953,0.181425
min,0.000000,1.011000e+12,2013.000000,202.000000,9.501000e+03,874.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,673.000000,2.000000,-88.263506,41.469868
25%,75226.500000,9.292160e+12,2015.000000,203.000000,1.400000e+05,3780.000000,45.000000,1.000000,1.000000,5.000000,...,1.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1104.000000,115.000000,-87.867421,41.712505
50%,167260.000000,1.534215e+13,2017.000000,204.000000,2.280000e+05,6100.000000,60.000000,2.000000,1.000000,6.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,3.000000,1364.000000,201.000000,-87.775311,41.873949
75%,269110.500000,2.413304e+13,2018.000000,234.000000,3.500000e+05,8547.000000,75.000000,3.000000,1.000000,7.000000,...,2.000000,1.000000,3.000000,2.000000,1.000000,3.000000,1888.000000,278.000000,-87.695410,42.015443
max,346285.000000,3.331402e+13,2020.000000,295.000000,1.905000e+06,48151.000000,184.000000,4.000000,6.000000,64.000000,...,2.000000,9.000000,3.000000,42.000000,7.000000,8.000000,4823.000000,366.000000,-87.524890,42.153979


In [14]:
bell_sales = bell_sales[(bell_sales.char_fbath<10)]

In [15]:
bell_sales.describe()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
count,286934.000000,2.869340e+05,286934.000000,286934.000000,2.869340e+05,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,...,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000
mean,171739.025525,1.640128e+13,2016.522514,221.866304,2.884854e+05,6966.508661,61.148958,1.987785,1.102023,6.242592,...,1.450508,0.359832,2.477984,1.496396,0.507249,3.170081,1594.126228,193.932660,-87.796878,41.854492
std,105161.074867,8.616728e+12,2.254254,31.761582,2.463901e+05,4900.273674,28.544073,0.817707,0.544856,1.510583,...,0.497545,0.566890,0.788935,0.670009,0.536805,1.616799,696.327036,99.887548,0.149954,0.181425
min,0.000000,1.011000e+12,2013.000000,202.000000,9.501000e+03,874.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,673.000000,2.000000,-88.263506,41.469868
25%,75226.250000,9.292160e+12,2015.000000,203.000000,1.400000e+05,3780.000000,45.000000,1.000000,1.000000,5.000000,...,1.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1104.000000,115.000000,-87.867421,41.712504
50%,167260.500000,1.534216e+13,2017.000000,204.000000,2.280000e+05,6100.000000,60.000000,2.000000,1.000000,6.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,3.000000,1364.000000,201.000000,-87.775310,41.873946
75%,269110.750000,2.413304e+13,2018.000000,234.000000,3.500000e+05,8547.000000,75.000000,3.000000,1.000000,7.000000,...,2.000000,1.000000,3.000000,2.000000,1.000000,3.000000,1888.000000,278.000000,-87.695410,42.015444
max,346285.000000,3.331402e+13,2020.000000,295.000000,1.905000e+06,48151.000000,184.000000,4.000000,6.000000,64.000000,...,2.000000,9.000000,3.000000,7.000000,7.000000,8.000000,4823.000000,366.000000,-87.524890,42.153979


In [16]:
bell_sales.char_bsmt_fin.value_counts()

3.0    191331
1.0     95572
2.0        31
Name: char_bsmt_fin, dtype: int64

In [17]:
bell_sales.char_heat.value_counts()

1.0    254383
2.0     31447
3.0       933
4.0       171
Name: char_heat, dtype: int64

In [18]:
bell_sales.char_air.value_counts()

1.0    157668
2.0    129266
Name: char_air, dtype: int64

In [19]:
bell_sales.char_frpl.value_counts()

0.0    195059
1.0     81859
2.0      8902
3.0       921
4.0       163
5.0        18
6.0         8
9.0         2
7.0         2
Name: char_frpl, dtype: int64

In [20]:
bell_sales_pin = bell_sales.meta_pin
bell_y = bell_sales.meta_sale_price
bell_X = bell_sales.drop(['Unnamed: 0','meta_pin','meta_sale_price'],axis=1)

In [21]:
bell_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286934 entries, 0 to 296501
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   meta_year              286934 non-null  float64
 1   meta_class             286934 non-null  int64  
 2   char_hd_sf             286934 non-null  float64
 3   char_age               286934 non-null  float64
 4   char_ext_wall          286934 non-null  float64
 5   char_roof_cnst         286934 non-null  float64
 6   char_rooms             286934 non-null  float64
 7   char_beds              286934 non-null  float64
 8   char_bsmt              286934 non-null  float64
 9   char_bsmt_fin          286934 non-null  float64
 10  char_heat              286934 non-null  float64
 11  char_air               286934 non-null  float64
 12  char_frpl              286934 non-null  float64
 13  char_attic_type        286934 non-null  float64
 14  char_fbath             286934 non-nu

meta_class is the cook county classification of each property based on age, squarefootage, number of stories, and if it is detached or attached to other homes. As age and square footage are seperate feautures, the useful info to extract from the class are the number of stories and if it is attached.

In [22]:
bell_X['one_story'] = [1 if x < 205 else 0 for x in bell_X['meta_class']]

In [23]:
bell_X.head()

,meta_year,meta_class,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,char_beds,char_bsmt,char_bsmt_fin,...,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude,one_story
0,2014.0,205,6534.0,112.0,1.0,1.0,6.0,3.0,1.0,3.0,...,1.0,2.0,1.0,0.0,3.0,1941.0,142.0,-88.139379,42.153953,0
1,2020.0,205,12012.0,127.0,1.0,1.0,7.0,4.0,1.0,3.0,...,1.0,3.0,2.0,0.0,3.0,2000.0,290.0,-88.140524,42.153505,0
2,2020.0,202,6699.0,80.0,1.0,1.0,4.0,2.0,1.0,3.0,...,1.0,3.0,1.0,0.0,4.0,807.0,171.0,-88.140083,42.153577,1
3,2020.0,205,9036.0,109.0,1.0,1.0,7.0,4.0,1.0,3.0,...,1.0,3.0,2.0,1.0,5.0,1408.0,351.0,-88.138970,42.153064,0
4,2017.0,203,8712.0,82.0,1.0,1.0,6.0,2.0,3.0,1.0,...,1.0,2.0,1.0,0.0,1.0,1366.0,178.0,-88.138708,42.153038,1


In [24]:
bell_X['row_house'] = [1 if ((x == 210) or (x==295)) else 0 for x in bell_X['meta_class']]

In [25]:
bell_X[bell_X.row_house==1]

,meta_year,meta_class,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,char_beds,char_bsmt,char_bsmt_fin,...,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude,one_story,row_house
174,2014.0,295,4594.0,34.0,3.0,1.0,6.0,3.0,1.0,1.0,...,3.0,1.0,1.0,3.0,1200.0,324.0,-88.128994,42.150849,0,1
175,2017.0,295,3693.0,37.0,3.0,1.0,6.0,3.0,1.0,1.0,...,3.0,1.0,1.0,3.0,1153.0,191.0,-88.128793,42.150788,0,1
176,2018.0,295,5319.0,38.0,3.0,1.0,6.0,3.0,1.0,1.0,...,3.0,1.0,1.0,3.0,1200.0,17.0,-88.128666,42.150749,0,1
556,2015.0,295,3485.0,54.0,2.0,1.0,8.0,4.0,3.0,1.0,...,3.0,2.0,0.0,1.0,1904.0,252.0,-88.142676,42.150920,0,1
877,2013.0,295,3678.0,13.0,3.0,1.0,7.0,3.0,1.0,3.0,...,3.0,3.0,1.0,3.0,2746.0,98.0,-88.134680,42.128004,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296101,2018.0,295,8000.0,17.0,1.0,1.0,6.0,3.0,1.0,1.0,...,3.0,2.0,1.0,3.0,1750.0,220.0,-87.557940,41.534021,0,1
296102,2014.0,295,8000.0,13.0,1.0,1.0,6.0,3.0,1.0,3.0,...,3.0,2.0,1.0,3.0,1750.0,353.0,-87.557936,41.533582,0,1
296103,2018.0,295,8000.0,17.0,1.0,1.0,6.0,3.0,1.0,3.0,...,3.0,2.0,1.0,3.0,1750.0,149.0,-87.557936,41.533582,0,1
296260,2015.0,295,4924.0,5.0,3.0,1.0,6.0,3.0,3.0,1.0,...,3.0,2.0,1.0,3.0,1987.0,155.0,-87.533869,41.516564,0,1


In [26]:
bell_X = bell_X.drop(columns='meta_class')

In [27]:
bell_X.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286934 entries, 0 to 296501
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   meta_year              286934 non-null  float64
 1   char_hd_sf             286934 non-null  float64
 2   char_age               286934 non-null  float64
 3   char_ext_wall          286934 non-null  float64
 4   char_roof_cnst         286934 non-null  float64
 5   char_rooms             286934 non-null  float64
 6   char_beds              286934 non-null  float64
 7   char_bsmt              286934 non-null  float64
 8   char_bsmt_fin          286934 non-null  float64
 9   char_heat              286934 non-null  float64
 10  char_air               286934 non-null  float64
 11  char_frpl              286934 non-null  float64
 12  char_attic_type        286934 non-null  float64
 13  char_fbath             286934 non-null  float64
 14  char_hbath             286934 non-nu

In [28]:
bell_X = bell_X.astype({'char_ext_wall':'category','char_roof_cnst':'category','char_bsmt':'category','char_bsmt_fin':'category','char_heat':'category','char_attic_type':'category','char_frpl':'category','char_gar1_size':'category'})

In [29]:
bell_X.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286934 entries, 0 to 296501
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   meta_year              286934 non-null  float64 
 1   char_hd_sf             286934 non-null  float64 
 2   char_age               286934 non-null  float64 
 3   char_ext_wall          286934 non-null  category
 4   char_roof_cnst         286934 non-null  category
 5   char_rooms             286934 non-null  float64 
 6   char_beds              286934 non-null  float64 
 7   char_bsmt              286934 non-null  category
 8   char_bsmt_fin          286934 non-null  category
 9   char_heat              286934 non-null  category
 10  char_air               286934 non-null  float64 
 11  char_frpl              286934 non-null  category
 12  char_attic_type        286934 non-null  category
 13  char_fbath             286934 non-null  float64 
 14  char_hbath          

In [30]:
bell_sales.describe()

,Unnamed: 0,meta_pin,meta_year,meta_class,meta_sale_price,char_hd_sf,char_age,char_ext_wall,char_roof_cnst,char_rooms,...,char_air,char_frpl,char_attic_type,char_fbath,char_hbath,char_gar1_size,char_bldg_sf,time_sale_day_of_year,geo_longitude,geo_latitude
count,286934.000000,2.869340e+05,286934.000000,286934.000000,2.869340e+05,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,...,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000,286934.000000
mean,171739.025525,1.640128e+13,2016.522514,221.866304,2.884854e+05,6966.508661,61.148958,1.987785,1.102023,6.242592,...,1.450508,0.359832,2.477984,1.496396,0.507249,3.170081,1594.126228,193.932660,-87.796878,41.854492
std,105161.074867,8.616728e+12,2.254254,31.761582,2.463901e+05,4900.273674,28.544073,0.817707,0.544856,1.510583,...,0.497545,0.566890,0.788935,0.670009,0.536805,1.616799,696.327036,99.887548,0.149954,0.181425
min,0.000000,1.011000e+12,2013.000000,202.000000,9.501000e+03,874.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,673.000000,2.000000,-88.263506,41.469868
25%,75226.250000,9.292160e+12,2015.000000,203.000000,1.400000e+05,3780.000000,45.000000,1.000000,1.000000,5.000000,...,1.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1104.000000,115.000000,-87.867421,41.712504
50%,167260.500000,1.534216e+13,2017.000000,204.000000,2.280000e+05,6100.000000,60.000000,2.000000,1.000000,6.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,3.000000,1364.000000,201.000000,-87.775310,41.873946
75%,269110.750000,2.413304e+13,2018.000000,234.000000,3.500000e+05,8547.000000,75.000000,3.000000,1.000000,7.000000,...,2.000000,1.000000,3.000000,2.000000,1.000000,3.000000,1888.000000,278.000000,-87.695410,42.015444
max,346285.000000,3.331402e+13,2020.000000,295.000000,1.905000e+06,48151.000000,184.000000,4.000000,6.000000,64.000000,...,2.000000,9.000000,3.000000,7.000000,7.000000,8.000000,4823.000000,366.000000,-87.524890,42.153979


In [31]:
bell_X_lr = pd.get_dummies(bell_X,drop_first=True)
bell_X_dt=  pd.get_dummies(bell_X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(bell_X_lr, bell_y, test_size=0.2, random_state=42)

In [33]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaling is needed for models that need to determine a 'distance" between data points. But for models, such as decesion trees, that do not require a distance, it is not required.Scaling should do no harm when it is not needed. 

First just try metrics for the mean value of the training data

In [34]:
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train,y_train)
dumb_reg.constant_



array([[288712.5194666]])

In [35]:
y_tr_pred = dumb_reg.predict(X_train)
y_te_pred = dumb_reg.predict(X_test)
mape(y_train, y_tr_pred), mape(y_test, y_te_pred)

(1.357666170774868, 1.3619960123352735)

Three metrics recommended by the International association of assessing officers based on the ratio y_pred/y_test are Coefficient of Dispersion (COD), Price Related Differential (PRD), and Coefficent of Price-Related Bias.

Coefficient of Dispersion (COD)is a measure of the variance of the ratio. It is multiplied by 100. Its value is the percentage variation of the ratio away from the median. It does not tell us if low priced homes are being treated differently than high priced homes.

In [36]:
def cod(y_pred,y_test):
    ratio = y_pred/y_test
    ratio_median = np.median(ratio)
    return 100*np.mean(np.abs(ratio-ratio_median))/ratio_median
cod(y_tr_pred,y_train), cod(y_te_pred,y_test)

(104.14807991409124, 103.92973962357863)

Just taking an average produces a variation of over 40% is the ratio.


Price Related Differential does tell us if low priced homes are being treated differently than high priced homes. It is the mean ratio divided by the sales price weighted mean ratio. If the ratio for large sales price homes is lower than the ratio for low sales price homes, then the mean ratio over the weighted mean ratio will be larger than 1.

In [37]:
def prd(y_pred,y_test):
    ratio = y_pred/y_test
    y_test_sum = np.sum(y_test)
    y_pred_sum = np.sum(y_pred)
    return np.mean(ratio)*y_test_sum/y_pred_sum
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

(2.1150600098949632, 2.1129302276872375)

The mean ratio is 22% higher than the weighted mean ratio, indicating that low proced homes are priced higher than their sales price compared to how high priced homes are treated.  This is a consequence that small variations in the predicated price of a low priced home makes a bigger change in the ratio.

Coefficent of price related bias (PRB) does a liner regression between the log2 of the price and the ratio. A negative number means that if one increases the price, the ratio goes down.

In [38]:
def prb(y_pred,y_test):
    proxy = 0.5*(y_pred + y_test)
    ln_value = pd.DataFrame(np.log2(proxy))
    ratio = y_pred/y_test
    ratio_median = np.median(ratio)
    pct_dif = (ratio - ratio_median)/ratio_median
    lr = LinearRegression()
    lr.fit(ln_value,pct_dif)
    return lr.coef_
prb(y_tr_pred,y_train), prb(y_te_pred,y_test)

(array([-2.70740791]), array([-2.7157076]))

Using the average price for all predictions has the consequence that doubling the price of the home means the ratio goes down by 178%.

Next we see how linear regression improves predicted prices.  

In [39]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [40]:
y_tr_pred = lm.predict(X_train)
y_te_pred = lm.predict(X_test)

In [41]:
mape(y_train, y_tr_pred), mape(y_test, y_te_pred)

(0.6698754125656607, 0.6758046276223649)

In [42]:
cod(y_tr_pred,y_train), cod(y_te_pred,y_test)

(64.77748019849841, 65.14805613175696)

variance is cut by more than half

In [43]:
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

(1.3590223882830588, 1.3628685279763662)

mean ratio is now only 7 percent higher than the weighted ratio.

In [44]:
prb(y_tr_pred,y_train), prb(y_te_pred,y_test)

/opt/anaconda3/envs/springboard/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred)

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred/y_train)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred/y_test)

In [ ]:
sns.scatterplot(x=np.log2(y_train),y=y_tr_pred/y_train)

In [ ]:
sns.scatterplot(x=np.log2(y_test),y=y_te_pred/y_test)

For linear regression, doubling the home price means the ratio goes down by almost 14%. Let's see how much this can be improved with ElasticNet which includes LinearRegression as a limit.

In [ ]:
en = ElasticNet()

In [ ]:
params = {'alpha':[0.0001,0.001,0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,3823.315949420067],'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}
en_gscv = GridSearchCV(en,params,n_jobs=-1,cv=5,scoring="neg_mean_absolute_percentage_error")

In [ ]:
en_gscv.fit(X_train, y_train)

In [ ]:
y_tr_pred = en_gscv.predict(X_train)
y_te_pred = en_gscv.predict(X_test)

In [ ]:
mape(y_train, y_tr_pred), mape(y_test, y_te_pred)

In [ ]:
cod(y_tr_pred,y_train), cod(y_te_pred,y_test)

In [ ]:
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

In [ ]:
prb(y_tr_pred,y_train), prb(y_te_pred,y_test)

In [ ]:
en_gscv.best_params_

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred)

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred/y_train)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred/y_test)

In [ ]:
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

In [ ]:
sns.scatterplot(x=np.log2(y_test),y=y_te_pred/y_test)

For linear regression, doubling the home price means the ratio goes down by almost 14%. Can this be improved with Random Forest?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bell_X_dt, bell_y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestRegressor(random_state=42)


In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_tr_pred = rf.predict(X_train)
y_te_pred = rf.predict(X_test)

In [ ]:
mape(y_train, y_tr_pred),mape(y_test, y_te_pred)


In [ ]:
cod(y_tr_pred,y_train), cod(y_te_pred,y_test)

variance compared to linear regression is reduced from 19% to 17%.

In [ ]:
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

prd is not inproved. Linear regression is better.

In [ ]:
prb(y_tr_pred,y_train), prb(y_te_pred,y_test)

prb is also not improved. Can we improve Random Forest with hyperparameter tuning?

In [ ]:
rfr= RandomForestRegressor(random_state=42)
params = {'n_estimators':[100,200,300,400,500],'criterion':["squared_error", "absolute_error", "poisson"],'max_features':["auto","sqrt","log2"]}
rfr_gscv = GridSearchCV(rfr,params,n_jobs=-1,cv=5,scoring="neg_mean_absolute_percentage_error")

In [ ]:
rfr_gscv.fit(X_train,y_train)

In [ ]:
y_tr_pred = rfr_gscv.predict(X_train)
y_te_pred = rfr_gscv.predict(X_test)

In [ ]:
mape(y_train, y_tr_pred),mape(y_test, y_te_pred)


In [ ]:
cod(y_tr_pred,y_train), cod(y_te_pred,y_test)

In [ ]:
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

In [ ]:
prb(y_tr_pred,y_train), prb(y_te_pred,y_test)

In [ ]:
rfr_gscv.best_params_

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred)

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred/y_train)

training data ratio is higher for low priced homes. The smallest priced home is valued at twice its sales price.

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred/y_test)

same bias in the test data

In [ ]:
sns.scatterplot(x=np.log2(y_train),y=y_tr_pred/y_train)

In [ ]:
sns.scatterplot(x=np.log2(y_test),y=y_te_pred/y_test)

One can also see the downward trend in when graphing against the log of the price. One high priced homes are valued at less than half of its sale price.

In [ ]:
gb = GradientBoostingRegressor(random_state=42)
params = {'n_estimators':[400,500,600],'learning_rate':[0.01,0.1,0.2],'loss':['squared_error', 'absolute_error'],'max_depth':[1,2,3]}
gb_gscv = GridSearchCV(gb,params,n_jobs=-1,cv=5,scoring="neg_mean_absolute_percentage_error")

In [ ]:
gb_gscv.fit(X_train,y_train)

In [ ]:
y_tr_pred = gb_gscv.predict(X_train)
y_te_pred = gb_gscv.predict(X_test)

In [ ]:
mape(y_train, y_tr_pred),mape(y_test, y_te_pred)

In [ ]:
cod(y_tr_pred,y_train), cod(y_te_pred,y_test)

In [ ]:
prd(y_tr_pred,y_train), prd(y_te_pred,y_test)

In [ ]:
prb(y_tr_pred,y_train), prb(y_te_pred,y_test)

In [ ]:
gb_gscv.best_params_

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred)

In [ ]:
sns.scatterplot(x=y_train,y=y_tr_pred/y_train)

In [ ]:
sns.scatterplot(x=y_test,y=y_te_pred/y_test)

In [ ]:
sns.scatterplot(x=np.log2(y_train),y=y_tr_pred/y_train)

In [ ]:
sns.scatterplot(x=np.log2(y_test),y=y_te_pred/y_test)